<a href="https://colab.research.google.com/github/gmgriebeler/cryptoprojects/blob/main/Jupiter_Trading_Volume_ATH_Alert_Dune_API_Integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTING LIBRARIES

In [38]:
import requests
import time
import os
import json
import shutil

SETTING PARAMETERS

In [39]:
DUNE_API_KEY = "8z6e753mdb9XDNVMwOo06io0T4KkfISC"
QUERY_ID = 5193365
DISCORD_WEBHOOK_URL = "https://discord.com/api/webhooks/1377240009652047962/cGLDakg2YqKy_hMJEsVeLiPPwA0qY7-25Kubuemhq3YR4u7fywG_0_eBUrV7AKmfCrFf"

headers = {
    "x-dune-api-key": DUNE_API_KEY
}

QUERY EXECUTION

In [40]:
execute_response = requests.post(
    f"https://api.dune.com/api/v1/query/{QUERY_ID}/execute",
    headers=headers
)
execution_id = execute_response.json()["execution_id"]
print(f"Query executed. Execution ID: {execution_id}")

Query executed. Execution ID: 01JWBJDPPDYSHR5TF0MK53Q3ZH


POLL FOR STATUS

In [41]:
while True:
    status_response = requests.get(
        f"https://api.dune.com/api/v1/execution/{execution_id}/status",
        headers=headers
    )
    state = status_response.json()["state"]
    print(f"Query state: {state}")

    if state == "QUERY_STATE_COMPLETED":
        break
    elif state in ["QUERY_STATE_FAILED", "QUERY_STATE_CANCELLED", "QUERY_STATE_ERRORED"]:
        raise Exception(f"Query failed with state: {state}")

    time.sleep(2)  # wait before checking again

Query state: QUERY_STATE_PENDING
Query state: QUERY_STATE_PENDING
Query state: QUERY_STATE_PENDING
Query state: QUERY_STATE_PENDING
Query state: QUERY_STATE_PENDING
Query state: QUERY_STATE_PENDING
Query state: QUERY_STATE_PENDING
Query state: QUERY_STATE_PENDING
Query state: QUERY_STATE_PENDING
Query state: QUERY_STATE_PENDING
Query state: QUERY_STATE_PENDING
Query state: QUERY_STATE_PENDING
Query state: QUERY_STATE_PENDING
Query state: QUERY_STATE_PENDING
Query state: QUERY_STATE_PENDING
Query state: QUERY_STATE_PENDING
Query state: QUERY_STATE_PENDING
Query state: QUERY_STATE_PENDING
Query state: QUERY_STATE_PENDING
Query state: QUERY_STATE_PENDING
Query state: QUERY_STATE_PENDING
Query state: QUERY_STATE_PENDING
Query state: QUERY_STATE_PENDING
Query state: QUERY_STATE_PENDING
Query state: QUERY_STATE_PENDING
Query state: QUERY_STATE_PENDING
Query state: QUERY_STATE_PENDING
Query state: QUERY_STATE_PENDING
Query state: QUERY_STATE_PENDING
Query state: QUERY_STATE_PENDING
Query stat

FETCH THE RESULTS

In [42]:
results_response = requests.get(
    f"https://api.dune.com/api/v1/execution/{execution_id}/results",
    headers=headers
)
results = results_response.json()
print(results)

{'execution_id': '01JWBJDPPDYSHR5TF0MK53Q3ZH', 'query_id': 5193365, 'is_execution_finished': True, 'state': 'QUERY_STATE_COMPLETED', 'submitted_at': '2025-05-28T13:48:31.82149Z', 'expires_at': '2025-08-26T13:51:46.827257Z', 'execution_started_at': '2025-05-28T13:50:35.926892Z', 'execution_ended_at': '2025-05-28T13:51:46.827256Z', 'result': {'rows': [{'volume': 2926425699.0544086, 'volume_rank': 76, 'week': '2025-05-26 00:00:00.000 UTC'}], 'metadata': {'column_names': ['week', 'volume', 'volume_rank'], 'column_types': ['timestamp(3) with time zone', 'double', 'bigint'], 'row_count': 1, 'result_set_bytes': 24, 'total_row_count': 1, 'total_result_set_bytes': 24, 'datapoint_count': 3, 'pending_time_millis': 124105, 'execution_time_millis': 70900}}}


EXTRACTING VALUES FROM RESULTS

In [43]:
row = results["result"]["rows"][0]  # assuming only 1 row is returned
week = row["week"][:10]
volume = row["volume"]
rank = row["volume_rank"]

COMPOSING AND SENDING DISCORD MESSAGE

In [44]:
discord_message = {
    "content": (
        f"Jupiter Total Volume Weekly Update\n"
        f"**Week of {week}**\n"
        f"Total Volume: **${volume:,.2f}**\n"
        f"Rank: **#{rank} all-time**"
    )
}

discord_response = requests.post(DISCORD_WEBHOOK_URL, json=discord_message)

if discord_response.status_code == 204:
    print("Message successfully sent to Discord!")
else:
    print(f"Failed to send message. Status: {discord_response.status_code}")
    print(discord_response.text)

Message successfully sent to Discord!


SAVING RESULTS TO JSON FILE

In [45]:
json_file_path = "dune_results.json"
with open(json_file_path, "w") as json_file:
    json.dump(results, json_file, indent=4)

print(f"JSON results saved to {json_file_path}")

downloads_path = os.path.join(os.environ["HOME"], "Downloads")
shutil.copy(json_file_path, downloads_path)

print(f"File copied to {downloads_path}")

JSON results saved to dune_results.json
File copied to /root/Downloads


EXPORTING JSON FILE

In [46]:
# from google.colab import files
# files.download("dune_results.json")